# Interactive seq2seq
Applying our trained encoder-decoder model so that it can produce and respond to dialogue in context. We will need to load the model, seq2seq_baseline.ckpt, which was trained on the NLP cluster with 2500 iterations and batch size of 1028.

In [2]:
import numpy as np
import tensorflow as tf
import warnings
import random
from tensorflow.python.layers.core import Dense

In [1]:
import sys
sys.path.append("../src/models/")
sys.path.append('../src/models/agents/')
sys.path.append('../src/data/')
from agent import Agent
from parse import FBParser

['', '/Users/Nicholas/GitHub/Negotiation/notebooks', '//Users/Nicholas/GitHub/Negotiation/cocoa-negotiation', '//Users/Nicholas/GitHub/Negotiation/cocoa-negotiation/fb-negotiation', '/usr/local/Cellar/python/3.6.5_1/Frameworks/Python.framework/Versions/3.6/lib/python36.zip', '/usr/local/Cellar/python/3.6.5_1/Frameworks/Python.framework/Versions/3.6/lib/python3.6', '/usr/local/Cellar/python/3.6.5_1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/lib-dynload', '/Users/Nicholas/Library/Python/3.6/lib/python/site-packages', '/usr/local/lib/python3.6/site-packages', '/Users/Nicholas/Thesis/Code/multiagent-particle-envs', '/Users/Nicholas/GitHub/Negotiation', '/usr/local/lib/python3.6/site-packages/IPython/extensions', '/Users/Nicholas/.ipython']


### Set global variables


In [19]:
train_iterations = 2500
learning_rate = 0.1
max_input_length = 6 # length of goals list
max_output_length = 20
unk_threshold = 20

### Initialize parser and agent

In [21]:
parser = FBParser(unk_threshold=unk_threshold)
print("Vocab size: {}".format(parser.vocab_size))

FileNotFoundError: [Errno 2] No such file or directory: '../../data/raw/train.txt'